In [1]:
%load_ext autoreload
%autoreload 2

Using Feature Enginering

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import RepeatedStratifiedKFold


In [2]:
df = pd.read_csv('combined_features.csv',index_col=0)
df.sort_index(inplace=True)

In [3]:
df

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread
time,,,,,,,,,
2021-10-04 08:00:00,1.659676,4.768259,4.786339,4.771101,1.107056,0.867368,0.543952,43.744151,102.684738
2021-10-04 09:00:00,1.659675,4.766889,4.776943,4.769641,1.032303,0.822687,0.844880,42.875897,102.612395
2021-10-04 10:00:00,1.660783,4.769485,4.775439,4.779400,1.121594,0.918496,2.310488,42.838238,104.369796
2021-10-04 11:00:00,1.664391,4.774242,4.771703,4.777858,1.000573,0.913061,0.528564,24.307401,103.491902
2021-10-04 12:00:00,1.664180,4.777072,4.769049,4.770852,0.963529,0.904516,0.586155,11.604788,103.568593
...,...,...,...,...,...,...,...,...,...
2022-09-23 13:00:00,1.579399,4.908354,4.900728,4.905692,1.341466,-0.357752,-2.098000,-26.144407,97.885000
2022-09-23 14:00:00,1.579414,4.905133,4.904967,4.903628,1.290696,-0.426738,-2.333600,-34.984305,97.701400
2022-09-23 15:00:00,1.579429,4.894920,4.903911,4.890514,1.061713,-0.552944,-3.514100,-43.456076,96.735900


In [4]:
#generate label for training
import numpy as np
def gen_labels(df,t,threshold):
    # Calculate % return on spread t hours later
    df['forward_return'] = df['spread'].diff(periods=t)/df['spread']
    
    #If the return is more than x%, we should have bought, and hence the label is (1)
    #If return is less than x%, we should have sold, and hence label is (-1)
    #If in between, do nothing (0)
    df['output'] = np.select([df['forward_return'] > threshold ,df['forward_return'] < -threshold],[1,-1])
    return df

df = gen_labels(df,t=24,threshold=0.02)
df

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread,forward_return,output
time,,,,,,,,,,,
2021-10-04 08:00:00,1.659676,4.768259,4.786339,4.771101,1.107056,0.867368,0.543952,43.744151,102.684738,NaN,0
2021-10-04 09:00:00,1.659675,4.766889,4.776943,4.769641,1.032303,0.822687,0.844880,42.875897,102.612395,NaN,0
2021-10-04 10:00:00,1.660783,4.769485,4.775439,4.779400,1.121594,0.918496,2.310488,42.838238,104.369796,NaN,0
2021-10-04 11:00:00,1.664391,4.774242,4.771703,4.777858,1.000573,0.913061,0.528564,24.307401,103.491902,NaN,0
2021-10-04 12:00:00,1.664180,4.777072,4.769049,4.770852,0.963529,0.904516,0.586155,11.604788,103.568593,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-23 13:00:00,1.579399,4.908354,4.900728,4.905692,1.341466,-0.357752,-2.098000,-26.144407,97.885000,-0.028656,-1
2022-09-23 14:00:00,1.579414,4.905133,4.904967,4.903628,1.290696,-0.426738,-2.333600,-34.984305,97.701400,-0.033097,-1
2022-09-23 15:00:00,1.579429,4.894920,4.903911,4.890514,1.061713,-0.552944,-3.514100,-43.456076,96.735900,-0.040152,-1


In [5]:
#train and test data
split = round(0.8*len(df))
train, test = df[:split],df[split:]
x_train = train[['vwap','SMA(5)','SMA(10)','12dayEWM','rsi','MACD','mom','mfi','spread']]
y_train = train[['output']]

x_test = test[['vwap','SMA(5)','SMA(10)','12dayEWM','rsi','MACD','mom','mfi','spread']]
y_test = test[['output']]

In [7]:

param_grid={'penalty':['l1','l2'], 'C':[1e-3, 1e-2, 1e-1, 1, 10]}

# Create a based model
lr = LogisticRegression()
# Instantiate the grid search model
model = GridSearchCV(estimator=LogisticRegression(solver="liblinear"), param_grid=param_grid) #helps to test different kinds of hype params
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/uti

In [10]:
print(classification_report(y_test,y_pred))
print(accuracy_score(y_pred, y_test))
print(model.best_params_)

              precision    recall  f1-score   support

          -1       0.92      0.76      0.83       103
           0       0.82      0.98      0.89       380
           1       1.00      0.25      0.40        76

    accuracy                           0.84       559
   macro avg       0.91      0.66      0.71       559
weighted avg       0.86      0.84      0.81       559

0.8407871198568873
{'C': 1, 'penalty': 'l2'}


In [11]:
joblib.dump(model, 'logreg.pkl')

['logreg.pkl']

Testing all hyperparams and solvers

In [19]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['l1','l2', 'none', 'elasticnet']
c_values = [1e-3, 1e-2, 1e-1, 1, 10]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
# param_grid={'penalty':['l1','l2'], 'C':[1e-3, 1e-2, 1e-1, 1, 10]}

# print(grid)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
# grid_search = GridSearchCV(estimator=LogisticRegression(solver="liblinear"), param_grid=param_grid)
grid_result = grid_search.fit(x_train, y_train.values.ravel())
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

Best: 0.811329 using {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'lbfgs'}
0.583140 (0.007648) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'sag'}
0.592979 (0.011950) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
0.642647 (0.013976) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'newton-cg'}
0.642048 (0.015968) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
0.613263 (0.014235) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'liblinear'}
0.634731 (0.017149) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'sag'}
0.634731 (0.017149) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}
0.811329 (0.028889) with: {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
0.804775 (0.025557) with: {'C': 0.001, 'penalty': 'none', 'solver': 'lbfgs'}
0.0000

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1350 fits failed out of a total of 3000.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Framewo